In [44]:
import numpy as np
import sys
import time
import pandas as pd
sys.path.append("../../")
sys.path.append("../../reco_utils/recommender/rlrmc/")

from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.dataset import movielens
from reco_utils.recommender.rlrmc.RLRMCdataset import RLRMCdataset 
from reco_utils.recommender.rlrmc.RLRMCalgorithm import RLRMCalgorithm 
# Pymanopt installation is required via
# pip install pymanopt 
from reco_utils.evaluation.python_evaluation import (
    rmse, mae
)

# import logging

# %load_ext autoreload
# %autoreload 2

In [45]:
print("Pandas version: {}".format(pd.__version__))
print("System version: {}".format(sys.version))

Pandas version: 0.25.3
System version: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [46]:
df = pd.read_csv('C:/Users/prath/Downloads/argos_r.csv', usecols=["userID", "itemID", "rating"])
df.head(10)

,userID,itemID,rating
0,17850.0,85123A,4
1,17850.0,71053,2
2,17850.0,84406B,5
3,17850.0,84029G,5
4,17850.0,84029E,3
5,17850.0,22752,3
6,17850.0,21730,1
7,17850.0,22633,2
8,17850.0,22632,4
9,13047.0,84879,4


In [47]:
df.dropna(subset=['userID'],how='all',inplace=True)
df["userID"] = df["userID"].apply(lambda x: str(x)[:-2])

In [48]:
df.head(10)

,userID,itemID,rating
0,17850,85123A,4
1,17850,71053,2
2,17850,84406B,5
3,17850,84029G,5
4,17850,84029E,3
5,17850,22752,3
6,17850,21730,1
7,17850,22633,2
8,17850,22632,4
9,13047,84879,4


In [49]:
df['userID'].astype(str).astype(int)

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: userID, Length: 406829, dtype: int32

In [50]:
df.dtypes

userID    object
itemID    object
rating     int64
dtype: object

In [51]:
df.shape

(406829, 3)

In [52]:
df.head(10)

,userID,itemID,rating
0,17850,85123A,4
1,17850,71053,2
2,17850,84406B,5
3,17850,84029G,5
4,17850,84029E,3
5,17850,22752,3
6,17850,21730,1
7,17850,22633,2
8,17850,22632,4
9,13047,84879,4


In [53]:
# Model parameters

# rank of the model, a positive integer (usually small), required parameter
rank_parameter = 10
# regularization parameter multiplied to loss function, a positive number (usually small), required parameter
regularization_parameter = 0.001
# initialization option for the model, 'svd' employs singular value decomposition, optional parameter
initialization_flag = 'svd' #default is 'random'
# maximum number of iterations for the solver, a positive integer, optional parameter
maximum_iteration = 100 #optional, default is 100
# maximum time in seconds for the solver, a positive integer, optional parameter
maximum_time = 300#optional, default is 1000

# Verbosity of the intermediate results
verbosity=0 #optional parameter, valid values are 0,1,2, default is 0
# Whether to compute per iteration train RMSE (and test RMSE, if test data is given)
compute_iter_rmse=True #optional parameter, boolean value, default is False

In [54]:
## Logging utilities. Please import 'logging' in order to use the following command. 
# logging.basicConfig(level=logging.INFO)

In [55]:
## If both validation and test sets are required
# train, validation, test = python_random_split(df,[0.6, 0.2, 0.2])

## If validation set is not required
train, test = python_random_split(df,[0.8, 0.2])

## If test set is not required
# train, validation = python_random_split(df,[0.8, 0.2])

## If both validation and test sets are not required (i.e., the complete dataset is for training the model)
# train = df

In [56]:

# data = RLRMCdataset(train=train, validation=validation, test=test)
data = RLRMCdataset(train=train, test=test) # No validation set
# data = RLRMCdataset(train=train, validation=validation) # No test set
# data = RLRMCdataset(train=train) # No validation or test set

In [57]:
model = RLRMCalgorithm(rank = rank_parameter,
                       C = regularization_parameter,
                       model_param = data.model_param,
                       initialize_flag = initialization_flag,
                       maxiter=maximum_iteration,
                       max_time=maximum_time)

In [58]:
start_time = time.time()

model.fit(data,verbosity=verbosity)

# fit_and_evaluate will compute RMSE on the validation set (if given) at every iteration
# model.fit_and_evaluate(data,verbosity=verbosity)

train_time = time.time() - start_time # train_time includes both model initialization and model training time. 

print("Took {} seconds for training.".format(train_time))

Took 6.569236993789673 seconds for training.


In [59]:

# Obtain prediction on the full test set
predictions_ndarr = model.predict(test['userID'].values,test['itemID'].values)

In [60]:
predictions_df = pd.DataFrame(data={"userID": test['userID'].values, "itemID":test['itemID'].values, "prediction":predictions_ndarr})

## Compute test RMSE 
eval_rmse = rmse(test, predictions_df)
## Compute test MAE 
eval_mae = mae(test, predictions_df)

print("RMSE:\t%f" % eval_rmse,
      "MAE:\t%f" % eval_mae, sep='\n')

RMSE:	3.089068
MAE:	2.233741


In [61]:
predictions_df.shape

(81366, 3)

In [62]:
predictions_df.head(20)

,userID,itemID,prediction
0,14375,22469,2.616205
1,17375,22325,3.606184
2,15498,84945,6.141017
3,16374,85078,-2.998823
4,13735,22489,2.848315
5,15644,84692,0.961378
6,16523,23581,0.904506
7,18125,22197,7.799440
8,16984,21929,2.355695
9,14911,15060B,9.246432


In [63]:
predictions_df.to_csv('RLRMC_final.csv')

In [64]:
import joblib
# save the model to disk
filename = 'RLRMC_final.sav'
joblib.dump(model, filename)

['RLRMC_final.sav']

In [65]:
thislist1 = ['14375']
thislist2 = ['22469']
model.predict(thislist1, thislist2)

array([3.0992273])

In [66]:
thislist1 = ['17375']
thislist2 = ['22325']
model.predict(thislist1, thislist2)

array([3.0187531])

In [67]:
result = predictions_df.loc[predictions_df['userID'] == '17375'].sort_values(by = 'prediction', ascending = False).head(5)

In [68]:
result = result['itemID'].reset_index(drop = True)
result

0     23201
1     22096
2    84313B
3     22325
4     17003
Name: itemID, dtype: object